In [32]:
import pandas as pd
import numpy as np
from pygbif import species
import os
import dotenv
from data_functions import *
from taxon_names_resolver import Resolver

In [33]:
dotenv.load_dotenv(".env")
#data_dir = os.getenv("DATA_PATH")
data_dir = 'G:\GITAR\\'

In [12]:
#read occurences/all_records.csv from data_dir
all_records = pd.read_csv(data_dir + "Database/occurrences/all_records.csv", low_memory=False)
master_usageKey = all_records['usageKey'].unique()

In [51]:
DAISIE_sources = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_literature_references.csv", low_memory=False)

In [54]:
#filter daisie sources to unique sourceid longref pairs
#convert id_sp_region to int, keep na
DAISIE_sources['id_sp_region'] = DAISIE_sources['id_sp_region'].fillna('nan')


DAISIE_sources = DAISIE_sources[['sourceid', 'longref','id_sp_region']].drop_duplicates()
#make a dictionary of sourceid:longref pairs
DAISIE_sources_dict = dict(zip(DAISIE_sources['sourceid'], DAISIE_sources['longref']))
DAISIE_region_sources_dict = dict(zip(DAISIE_sources['id_sp_region'], DAISIE_sources['longref']))

In [15]:
#read in DAISIE species
DAISIE_species = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_taxon.csv", low_memory=False)

In [16]:
#create column genus_species for DAISIE
DAISIE_species['genus_species'] = DAISIE_species['genus'] + ' ' + DAISIE_species['species']
#create column source that is sourceid mapped to daiise_sources_dict
DAISIE_species['source'] = DAISIE_species['sourceid'].map(DAISIE_sources_dict)



In [20]:
#replace species with genus_species in daiise_species and remove genus
DAISIE_species['species'] = DAISIE_species['genus_species']
DAISIE_species = DAISIE_species.drop(columns=['genus_species', 'genus'])


In [ ]:
#foo = DAISIE_species.head().copy()
gbif_species_match(DAISIE_species)

In [14]:
#for species where usagekey is na, make usagekey "XX" + "DAI" "first 3 letters of first word of species" + "first three letters of second word of species"
#unless spece is blank then make it "XXDAI" plus first three letters of taxon_group plus sourceid passed to int  to string

#DAISIE_species['usageKey'] = np.where(DAISIE_species['usageKey'].isna(), 'XXDAI' + DAISIE_species['species'].str.split().str[0].str[:3] + DAISIE_species['species'].str.split().str[1].str[:3], DAISIE_species['usageKey'])
# Fill NaN values in 'species' column (if any) with an empty string or any desired default value

# Fill 'usageKey' based on conditions
condition1 = DAISIE_species['usageKey'].isna() & DAISIE_species['species'].notna()
condition2 = DAISIE_species['usageKey'].isna() & DAISIE_species['species'].isna()

DAISIE_species['usageKey'] = np.where(
    condition1,
    'XXDAI' + DAISIE_species['species'].str.split().str[0].str[:3] + DAISIE_species['species'].str.split().str[1].str[:3],
    DAISIE_species['usageKey']
)

DAISIE_species['usageKey'] = np.where(
    condition2,
    'XXDAI' + DAISIE_species['taxon_group'].str[:3] + DAISIE_species['sourceid'].astype(str),
    DAISIE_species['usageKey']
)

# Now, check for NA values in 'usageKey'
na_usage_key = DAISIE_species[DAISIE_species['usageKey'].isna()]
print(na_usage_key)


Empty DataFrame
Columns: [idspecies, species, authority, subtaxon, subtaxon_rank, subtaxon_authority, taxon_group, phylum, class, ordo, family, ecofunct_group, sourceid, source, usageKey, scientificName, canonicalName, rank, confidence, matchType]
Index: []


In [15]:
#write daisie species to csv in raw folder as daisie_species_add_gbif.csv
DAISIE_species.to_csv(data_dir + "Database/DAISIE data/raw/DAISIE_species_add_gbif.csv", index=False)


In [13]:
#read daisie species from csv in raw folder as daisie_species_add_gbif.csv
#DAISIE_species = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_species_add_gbif.csv", low_memory=False)
#print speceis with na usagekey
#DAISIE_species[DAISIE_species['usageKey'].isna()]



#print rows where usageKey is na and species is na 
DAISIE_species[(DAISIE_species['usageKey'].isna()) & (DAISIE_species['species'].isna())]


,idspecies,species,authority,subtaxon,subtaxon_rank,subtaxon_authority,taxon_group,phylum,class,ordo,family,ecofunct_group,sourceid,source,usageKey,scientificName,canonicalName,rank,confidence,matchType
503,2271,NaN,Tratt.,NaN,agg.,NaN,Magnoliophyta,Magnoliophyta,Liliopsida,Asparagales,Agavaceae,NaN,26.0,"Greuter W, Burdet HM, Long G (eds) (1984) Med ...",NaN,NaN,NaN,NaN,NaN,NaN
619,2737,NaN,NaN,hyb.,hyb.,NaN,Magnoliophyta,Magnoliophyta,Liliopsida,Poales,Poaceae,NaN,46.0,"Greuter W, Raus Th (1995) Med - Checklist Notu...",NaN,NaN,NaN,NaN,NaN,NaN
930,3928,NaN,NaN,NaN,hyb.,NaN,Magnoliophyta,Magnoliophyta,Liliopsida,Poales,Poaceae,NaN,6.0,Davis PH et al (eds) (1967) Flora of Turkey an...,NaN,NaN,NaN,NaN,NaN,NaN
931,3929,NaN,NaN,NaN,hyb.,NaN,Magnoliophyta,Magnoliophyta,Liliopsida,Poales,Poaceae,NaN,6.0,Davis PH et al (eds) (1967) Flora of Turkey an...,NaN,NaN,NaN,NaN,NaN,NaN
1089,4456,NaN,Sieb. & Zucc.,NaN,agg.,NaN,Magnoliophyta,Magnoliophyta,Liliopsida,Poales,Poaceae,NaN,26.0,"Greuter W, Burdet HM, Long G (eds) (1984) Med ...",NaN,NaN,NaN,NaN,NaN,NaN
1604,6197,NaN,NaN,NaN,agg.,NaN,Magnoliophyta,Magnoliophyta,Rosopsida,Caryophyllales,Amaranthaceae,NaN,6.0,Davis PH et al (eds) (1967) Flora of Turkey an...,NaN,NaN,NaN,NaN,NaN,NaN
2356,9174,NaN,Planchon,NaN,agg.,NaN,Magnoliophyta,Magnoliophyta,Rosopsida,Vitales,Vitaceae,NaN,19.0,"Tutin TG et al (eds) (1980) Flora Europaea, vo...",NaN,NaN,NaN,NaN,NaN,NaN
2428,9433,NaN,L.,NaN,agg.,NaN,Magnoliophyta,Magnoliophyta,Rosopsida,Geraniales,Geraniaceae,NaN,6.0,Davis PH et al (eds) (1967) Flora of Turkey an...,NaN,NaN,NaN,NaN,NaN,NaN
2546,9774,NaN,L.,NaN,agg.,NaN,Magnoliophyta,Magnoliophyta,Rosopsida,Myrtales,Onagraceae,NaN,6.0,Davis PH et al (eds) (1967) Flora of Turkey an...,NaN,NaN,NaN,NaN,NaN,NaN
3650,14374,NaN,NaN,NaN,agg.,NaN,Magnoliophyta,Magnoliophyta,Rosopsida,Rosales,Rosaceae,NaN,6.0,Davis PH et al (eds) (1967) Flora of Turkey an...,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
#find unique idspecies usageKey pairs
DAISIE_species_usageKey = DAISIE_species[['idspecies', 'usageKey']].drop_duplicates()
#make a dictionary of idspecies:usageKey pairs
DAISIE_species_usageKey_dict = dict(zip(DAISIE_species_usageKey['idspecies'], DAISIE_species_usageKey['usageKey']))


In [50]:
#DAISIE_donor_areas
donor_area = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_donor_area.csv", low_memory=False)
#select idspecies, region, and sourceid
#if sourceid is null, set to 'DAISIE', otherwise map to sourceid:longref dictionary
donor_area['source'] = np.where(donor_area['sourceid'].isnull(), 'DAISIE', donor_area['sourceid'].map(DAISIE_sources_dict))
#create column usageKey by mapping idspecies to usageKey in DAISIE_species_usageKey_dict
donor_area['usageKey'] = donor_area['idspecies'].map(DAISIE_species_usageKey_dict)
#drop idspecies and sourceid columns
donor_area = donor_area.drop(columns=[ 'sourceid'])
#rename idspecies to DAISIE_idspecies
donor_area = donor_area.rename(columns={'idspecies': 'DAISIE_idspecies'})

#drop duplicates
donor_area = donor_area.drop_duplicates()
#rename region to donor_region
donor_area = donor_area.rename(columns={'region': 'donor_region'})
#write to csv in DAIISIE data/
donor_area.to_csv(data_dir + "Database/DAISIE data/DAISIE_donor_area.csv", index=False)


In [51]:
#daisie habitat
habitat = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_habitat.csv", low_memory=False)
#if sourceid is null, set to 'DAISIE', otherwise map to sourceid:longref dictionary
habitat['source'] = np.where(habitat['sourceid'].isnull(), 'DAISIE', habitat['sourceid'].map(DAISIE_sources_dict))
#create column usageKey by mapping idspecies to usageKey in DAISIE_species_usageKey_dict
habitat['usageKey'] = habitat['idspecies'].map(DAISIE_species_usageKey_dict)
#drop  sourceid columns
habitat = habitat.drop(columns=[ 'sourceid'])
#rename idspecies to DAISIE_idspecies
habitat = habitat.rename(columns={'idspecies': 'DAISIE_idspecies'})
#write to csv in DAIISIE data/
habitat.to_csv(data_dir + "Database/DAISIE data/DAISIE_habitat.csv", index=False)



In [52]:
#daisie pathways
pathways = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_pathways.csv", low_memory=False)
#if sourceid is null, set to 'DAISIE', otherwise map to sourceid:longref dictionary
pathways['source'] = np.where(pathways['sourceid'].isnull(), 'DAISIE', pathways['sourceid'].map(DAISIE_sources_dict))
#create column usageKey by mapping idspecies to usageKey in DAISIE_species_usageKey_dict
pathways['usageKey'] = pathways['idspecies'].map(DAISIE_species_usageKey_dict)
#drop  sourceid columns
pathways = pathways.drop(columns=[ 'sourceid'])
#rename idspecies to DAISIE_idspecies
pathways = pathways.rename(columns={'idspecies': 'DAISIE_idspecies'})
#drop id_sp_reg
pathways = pathways.drop(columns=[ 'id_sp_region'])

#write to csv in DAIISIE data/
pathways.to_csv(data_dir + "Database/DAISIE data/DAISIE_pathways.csv", index=False)


In [53]:
#daisie vectors
vectors = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_vectors.csv", low_memory=False)
#if sourceid is null, set to 'DAISIE', otherwise map to sourceid:longref dictionary
vectors['source'] = np.where(vectors['sourceid'].isnull(), 'DAISIE', vectors['sourceid'].map(DAISIE_sources_dict))
#create column usageKey by mapping idspecies to usageKey in DAISIE_species_usageKey_dict
vectors['usageKey'] = vectors['idspecies'].map(DAISIE_species_usageKey_dict)
#drop  sourceid columns
vectors = vectors.drop(columns=[ 'sourceid'])
#rename idspecies to DAISIE_idspecies
vectors = vectors.rename(columns={'idspecies': 'DAISIE_idspecies'})
#drop id_sp_reg
vectors = vectors.drop(columns=[ 'id_sp_region'])
#write to csv in DAIISIE data/
vectors.to_csv(data_dir + "Database/DAISIE data/DAISIE_vectors.csv", index=False)


In [54]:
#vernacular names
vernacular = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_vernacular_names.csv", low_memory=False)
#if sourceid is null, set to 'DAISIE', otherwise map to sourceid:longref dictionary
vernacular['source'] = np.where(vernacular['sourceid'].isnull(), 'DAISIE', vernacular['sourceid'].map(DAISIE_sources_dict))
#create column usageKey by mapping idspecies to usageKey in DAISIE_species_usageKey_dict
vernacular['usageKey'] = vernacular['idspecies'].map(DAISIE_species_usageKey_dict)
#drop  sourceid columns
vernacular = vernacular.drop(columns=[ 'sourceid'])
#rename idspecies to DAISIE_idspecies
vernacular = vernacular.rename(columns={'idspecies': 'DAISIE_idspecies'})
#write to csv in DAIISIE data/
vernacular.to_csv(data_dir + "Database/DAISIE data/DAISIE_vernacular_names.csv", index=False)


In [46]:
#remove duplicated index values from distribution
distribution = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_distribution.csv", low_memory=False)
distribution = distribution.drop_duplicates()
distribution.to_csv(data_dir + "Database/DAISIE data/raw/DAISIE_distribution.csv", index=False)


0

In [48]:
DAISIE_region_sources_dict

{nan: 'Anon. (????) Flora of the World',
 nan: 'Anon. (????) Genera Palmarum',
 nan: 'Anon. (????) INDEX Kewensis, vol 2',
 7639.0: 'Tutin TG, Heywood et al (eds) (1964-1980) Flora Europaea. Cambridge University Press, Cambridge',
 7776.0: 'Tutin TG, Heywood et al (eds) (1964-1980) Flora Europaea. Cambridge University Press, Cambridge',
 7777.0: 'Tutin TG, Heywood et al (eds) (1964-1980) Flora Europaea. Cambridge University Press, Cambridge',
 7803.0: 'Tutin TG, Heywood et al (eds) (1964-1980) Flora Europaea. Cambridge University Press, Cambridge',
 7804.0: 'Tutin TG, Heywood et al (eds) (1964-1980) Flora Europaea. Cambridge University Press, Cambridge',
 7805.0: 'Tutin TG, Heywood et al (eds) (1964-1980) Flora Europaea. Cambridge University Press, Cambridge',
 7806.0: 'Tutin TG, Heywood et al (eds) (1964-1980) Flora Europaea. Cambridge University Press, Cambridge',
 7807.0: 'Tutin TG, Heywood et al (eds) (1964-1980) Flora Europaea. Cambridge University Press, Cambridge',
 7810.0: 'Tut

In [56]:
#daisie distribution
distribution = pd.read_csv(data_dir + "Database/DAISIE data/raw/DAISIE_distribution.csv", low_memory=False)
#if sourceid is null, set to 'DAISIE', otherwise map to sourceid:longref dictionary
#distribution['source'] = np.where(distribution['sourceid'].isnull(), 'DAISIE', distribution['sourceid'].map(DAISIE_sources_dict))
#create column usageKey by mapping idspecies to usageKey in DAISIE_species_usageKey_dict
#distribution['usageKey'] = distribution['idspecies'].map(DAISIE_species_usageKey_dict)
#drop  sourceid columns
#distribution = distribution.drop(columns=[ 'sourceid'])
#rename idspecies to DAISIE_idspecies
#create column source that is id_sp_region mapped to daiise_region_sources_dict
#reset index to get rid of duplicate index
DAISIE_region_sources_dict['nan'] = 'DAISIE'
distribution = distribution.drop_duplicates()

#if id_sp_region is null, set to 'DAISIE', otherwise map to id_sp_region:longref dictionary
distribution['source'] = np.where(distribution['id_sp_region'].isnull(), 'DAISIE', distribution['id_sp_region'].map(DAISIE_region_sources_dict))


distribution = distribution.rename(columns={'idspecies': 'DAISIE_idspecies'})
#write to csv in DAIISIE data/
distribution.to_csv(data_dir + "Database/DAISIE data/DAISIE_distribution.csv", index=False)
